# Downloading parcellation and probability maps

`siibra` can facilitate structured, automatized and reprodicible download of parcellation maps. In this example, we demonstrate how to download all maximum probability and probabilistic maps of *Julich-Brain* - more then 300 structures available in different spaces. 

As often, we start by loading the human atlas.

In [ ]:
import siibra
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS

Maps are delineations of brain regions in a particular coordinate space. We access regions from a parcellation object, and request maps by specifying the desired target space. In the simple case, we just fetch the first of possibly several labelled maps offered by the parcellation:

In [ ]:
# request the parcellation object
julichbrain = atlas.get_parcellation('julich')

# request the map object for the desired space and type
lmap = julichbrain.get_map(space="mni152",maptype="labelled")

# fetch the (first available) actual image volume of this map. 
img = lmap.fetch()
print(type(img))

To fetch all available maps from the given parcellation, we iterate in two loops: 

- First loop: all available map types 
- Second loop: All maps that the parcellation defines for a given type

In [ ]:
targetspace = atlas.get_space('mni152')

for maptype in siibra.MapType:
    
    print(f"Map type: {maptype.name}")
    
    parcellation_map = julichbrain.get_map(targetspace,maptype)
    fname_head = f"{julichbrain.key}-{targetspace.key}-{maptype.name}"

    for mapindex,img in enumerate(parcellation_map.fetch_iter()):
        
        # Check if this mapindex represents a region or not. 
        # For Julich-Brain, this will be the case for the continuous map type, 
        # which represents the probability maps.
        try:
            region = parcellation_map.decode_label(mapindex=mapindex)
            fname_tail = f"{region.key}"
        except Exception:
            fname_tail = f"{mapindex:03d}"

        fname = siibra.commons.snake2camel(f"{fname_head}-{fname_tail}.nii.gz")
        print(f" - {fname}")
        img.to_filename(fname)
        
        # we stop after 5 maps in this example
        if mapindex>4:
            break